In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('/content/diabetes.csv')

In [4]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
df.shape

(768, 9)

In [6]:
df.isnull().sum()

,0
Pregnancies,0
Glucose,0
BloodPressure,0
SkinThickness,0
Insulin,0
BMI,0
DiabetesPedigreeFunction,0
Age,0
Outcome,0


In [7]:
df.corr()['Outcome'].sort_values(ascending=False)

,Outcome
Outcome,1.000000
Glucose,0.466581
BMI,0.292695
Age,0.238356
Pregnancies,0.221898
DiabetesPedigreeFunction,0.173844
Insulin,0.130548
SkinThickness,0.074752
BloodPressure,0.065068


In [8]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [63]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout


In [12]:
model = Sequential()
model.add(Dense(32,activation='relu', input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [13]:
model.fit(X_train, y_train,batch_size=32,epochs = 100, validation_data=(X_test, y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7014 - loss: 0.6085 - val_accuracy: 0.6623 - val_loss: 0.6136
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7092 - loss: 0.5783 - val_accuracy: 0.6948 - val_loss: 0.5882
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7220 - loss: 0.5536 - val_accuracy: 0.7208 - val_loss: 0.5686
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7497 - loss: 0.5225 - val_accuracy: 0.7208 - val_loss: 0.5552
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7490 - loss: 0.5110 - val_accuracy: 0.7143 - val_loss: 0.5448
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7635 - loss: 0.4972 - val_accuracy: 0.7338 - val_loss: 0.5376
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7808 - loss: 0.4789 - val_accuracy: 0.7338 - val_loss: 0.5335
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7805 - loss: 0.4805 - val_accuracy: 0.7338 - 

# To select appropriate optimizer

In [14]:
!pip install -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.0 MB/s eta 0:00:00


In [17]:
import kerastuner as kt

In [18]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(32,activation='relu', input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  optimizer = hp.Choice('optimizer', values = ['adam','sgd','rmsprop'])
  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

  return model


In [19]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

In [20]:
tuner.search(X_train,y_train,epochs=5, validation_data=(X_test,y_test), verbose=1)

Trial 3 Complete [00h 00m 03s]
val_accuracy: 0.7402597665786743

Best val_accuracy So Far: 0.7402597665786743
Total elapsed time: 00h 00m 07s


In [21]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [22]:
model = tuner.get_best_models(num_models=1)[0]

In [23]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
model.fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.7229 - loss: 0.5600 - val_accuracy: 0.7273 - val_loss: 0.5300
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7099 - loss: 0.5416 - val_accuracy: 0.7338 - val_loss: 0.5168
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7742 - loss: 0.4783 - val_accuracy: 0.7338 - val_loss: 0.5101
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7580 - loss: 0.4950 - val_accuracy: 0.7532 - val_loss: 0.5049
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7746 - loss: 0.4701 - val_accuracy: 0.7532 - val_loss: 0.5020
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7829 - loss: 0.4618 - val_accuracy: 0.7532 - val_loss: 0.4985
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7539 - loss: 0.4816 - val_accuracy: 0.7662 - val_loss: 0.4984
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7981 - loss: 0.4517 - val_accuracy: 0.77

# To find the correct number of neurons

In [29]:
def build_model(hp):
  model = Sequential()
  units = hp.Int('units', min_value=8,max_value=128,step=8)

  model.add(Dense(units=units,activation ='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [30]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory = 'mydir',
                        project_name = 'Units')

In [31]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7207792401313782

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 13s


In [32]:
tuner.get_best_hyperparameters()[0].values

{'units': 88}

In [34]:
model = tuner.get_best_models(num_models=1)[0]

In [37]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.8629 - loss: 0.3484 - val_accuracy: 0.7403 - val_loss: 0.5751
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8238 - loss: 0.3818 - val_accuracy: 0.7403 - val_loss: 0.5743
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8288 - loss: 0.3658 - val_accuracy: 0.7403 - val_loss: 0.5767
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8567 - loss: 0.3413 - val_accuracy: 0.7338 - val_loss: 0.5812
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8464 - loss: 0.3350 - val_accuracy: 0.7338 - val_loss: 0.5776
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8335 - loss: 0.3503 - val_accuracy: 0.7273 - val_loss: 0.5802
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8482 - loss: 0.3531 - val_accuracy: 0.7338 - val_loss: 0.5762
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8624 - loss: 0.3283 - val_accuracy: 0.7

# To select number of layers

In [41]:
def build_model(hp):
  model = Sequential()

  model.add(Dense(72,activation='relu',input_dim=8))

  for i in range (hp.Int('num_layers',min_value=1,max_value=10)):
    model.add(Dense(72,activation='relu'))

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [42]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory = 'mydir',
                        project_name = 'Layers')

In [43]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 08s]
val_accuracy: 0.7532467246055603

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 25s


In [44]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 5}

In [46]:
model = tuner.get_best_models(num_models=1)[0]

In [47]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.8012 - loss: 0.4295 - val_accuracy: 0.7662 - val_loss: 0.5259
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8108 - loss: 0.3909 - val_accuracy: 0.7468 - val_loss: 0.5485
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8277 - loss: 0.3878 - val_accuracy: 0.7727 - val_loss: 0.5634
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8492 - loss: 0.3555 - val_accuracy: 0.7532 - val_loss: 0.5788
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8619 - loss: 0.3287 - val_accuracy: 0.7857 - val_loss: 0.5620
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8741 - loss: 0.3085 - val_accuracy: 0.7532 - val_loss: 0.6287
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8611 - loss: 0.3504 - val_accuracy: 0.7662 - val_loss: 0.5926
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8593 - loss: 0.3319 - val_accuracy: 0.75

In [67]:
def build_model(hp):
  model =Sequential()

  counter = 0
  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    if counter == 0:
      model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
                      activation = hp.Choice('choice'+str(i), values=['relu','tanh','sigmoid']),
                      input_dim=8)),
      model.add(Dropout(hp.Choice('dropout'+str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:
        model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
                      activation = hp.Choice('choice'+str(i), values=['relu','tanh','sigmoid']),
                      )),
        model.add(Dropout(hp.Choice('dropout'+str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))


        counter += 1

  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','sgd','adam','nadam','adadelta']),
                loss = 'binary_crossentropy',
                metrics=['accuracy'])
  return model

In [68]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory = 'mydir',
                        project_name = 'final1')

In [69]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 21s


In [70]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1,
 'units0': 72,
 'choice0': 'relu',
 'dropout0': 0.2,
 'optimizer': 'rmsprop',
 'units1': 80,
 'choice1': 'tanh',
 'dropout1': 0.5,
 'units2': 120,
 'choice2': 'relu',
 'dropout2': 0.5,
 'units3': 80,
 'choice3': 'tanh',
 'dropout3': 0.6,
 'units4': 96,
 'choice4': 'relu',
 'dropout4': 0.3,
 'units5': 72,
 'choice5': 'tanh',
 'dropout5': 0.6,
 'units6': 96,
 'choice6': 'relu',
 'dropout6': 0.6,
 'units7': 48,
 'choice7': 'tanh',
 'dropout7': 0.8,
 'units8': 56,
 'choice8': 'sigmoid',
 'dropout8': 0.1,
 'units9': 104,
 'choice9': 'sigmoid',
 'dropout9': 0.4}

In [71]:
model = tuner.get_best_models(num_models=1)[0]

In [72]:
model.fit(X_train, y_train, epochs=200, initial_epoch=5, validation_data=(X_test, y_test))

Epoch 6/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7250 - loss: 0.5122 - val_accuracy: 0.7792 - val_loss: 0.4968
Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7649 - loss: 0.4887 - val_accuracy: 0.7857 - val_loss: 0.4916
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7735 - loss: 0.4700 - val_accuracy: 0.7727 - val_loss: 0.4883
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7736 - loss: 0.4643 - val_accuracy: 0.7532 - val_loss: 0.4871
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7719 - loss: 0.4659 - val_accuracy: 0.7662 - val_loss: 0.4860
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7682 - loss: 0.4632 - val_accuracy: 0.7532 - val_loss: 0.4868
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7900 - loss: 0.4674 - val_accuracy: 0.7532 - val_loss: 0.4864
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8083 - loss: 0.4254 - val_accuracy: 0.746

In [73]:
import shutil

# Replace 'your_directory_name' with the directory path in Colab
directory_to_zip = "/content/mydir"
output_zip = "directory.zip"  # Output file name

# Compress the directory
shutil.make_archive("directory", 'zip', directory_to_zip)


'/content/directory.zip'

In [74]:
from google.colab import files

# Download the zip file to your system
files.download(output_zip)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>